In [1]:
using LinearAlgebra
using Dates

using Plots
using BenchmarkTools

BLAS.set_num_threads(1)

include("src/io.jl")
include("src/distances.jl")
include("src/optimizer.jl")
include("src/network.jl")
include("src/symmfunctions.jl")
include("src/base.jl")
include("src/pretraining.jl")

# Initialize the parameters
globalParms, MCParms, NNParms, preTrainParms, systemParmsList = parametersInit()

# Initialize the input data
inputs = inputInit(globalParms, NNParms, preTrainParms, systemParmsList)
if globalParms.mode == "training"
    model, opt, refRDFs = inputs
else
    model = inputs
end

Running ML-IMC in the training mode.
Building a model...
Chain(Dense(18 => 20, relu; bias=false), Dense(20 => 20, relu; bias=false), Dense(20 => 1; bias=false))
   Number of layers: 4 
   Number of neurons in each layer: [18, 20, 20, 1]


(Chain(Dense(18 => 20, relu; bias=false), Dense(20 => 20, relu; bias=false), Dense(20 => 1; bias=false)), AMSGrad(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}()), Any[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.004, 1.005, 1.006, 1.005, 1.006, 1.006, 1.007, 1.007, 1.007, 1.007], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.003, 1.003, 1.003, 1.004, 1.004, 1.004, 1.005, 1.005, 1.004, 1.005], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.004, 1.004, 1.004, 1.005, 1.005, 1.005, 1.005, 1.005, 1.005, 1.005], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.004, 1.004, 1.005, 1.004, 1.006, 1.003, 1.004, 1.004, 1.005, 1.004]])

In [2]:
    println("Using the following symmetry functions as the neural input for each atom:")
    if NNParms.G2Functions != []
        println("    G2 symmetry functions:")
        println("    eta, Å^-2; rcutoff, Å; rshift, Å")
        for G2Function in NNParms.G2Functions
            println("       ", G2Function)
        end
    end
    if NNParms.G9Functions != []
        println("    G9 symmetry functions:")
        println("    eta, Å^-2; lambda; zeta; rcutoff, Å; rshift, Å")
        for G9Function in NNParms.G9Functions
            println("       ", G9Function)
        end
    end
    println("Maximum cutoff distance: $(NNParms.maxDistanceCutoff) Å")

Using the following symmetry functions as the neural input for each atom:
    G2 symmetry functions:
    eta, Å^-2; rcutoff, Å; rshift, Å
       G2(0.003, 6.0, 0.0)
       G2(0.03, 6.0, 0.0)
       G2(0.1, 6.0, 0.0)
       G2(0.2, 6.0, 0.0)
       G2(0.5, 6.0, 3.35)
       G2(1.0, 6.0, 3.35)
       G2(2.0, 6.0, 3.35)
       G2(5.0, 6.0, 3.35)
       G2(0.5, 6.0, 4.5)
       G2(1.0, 6.0, 4.5)
       G2(2.0, 6.0, 4.5)
       G2(5.0, 6.0, 4.5)
       G2(0.5, 9.0, 7.75)
       G2(1.0, 9.0, 7.75)
       G2(2.0, 9.0, 7.75)
       G2(5.0, 9.0, 7.75)
    G9 symmetry functions:
    eta, Å^-2; lambda; zeta; rcutoff, Å; rshift, Å
       G9(0.125, 1, 1.0, 6.0, 0.0)
       G9(0.125, -1, 1.0, 6.0, 0.0)
Maximum cutoff distance: 9.0 Å


In [3]:
systemParms = systemParmsList[1];

In [4]:
traj = readXTC(systemParms)
frame = read_step(traj, 1)
coordinates = positions(frame)
box = lengths(UnitCell(frame));

In [5]:
distanceMatrix = buildDistanceMatrix(frame);

In [6]:
G2Matrix = buildG2Matrix(distanceMatrix, NNParms.G2Functions);

In [7]:
@btime buildG2Matrix($distanceMatrix, $NNParms.G2Functions);

  31.424 ms (514 allocations: 2.13 MiB)


In [8]:
pointIndex = 1;

In [9]:
distanceVector1 = distanceMatrix[:, pointIndex]

512-element Vector{Float64}:
  0.0
  7.091892808138942
 15.854950124730799
 17.4607272015029
 12.027773352854163
  6.82341543296718
 13.098859274949335
 11.326277833108653
 19.656198560684448
 15.23458176314022
  9.032409199465086
 15.041669488657645
 15.268684775800134
  ⋮
 18.560362730062472
 14.736974614013517
 20.77740909565999
 16.196676724150503
 15.990429100693698
 11.823270083877945
 13.597919877031464
 13.102531054745459
 14.870004024270196
  9.683171701468305
 15.980994682509369
 20.827134860816255

In [10]:
positions(frame)[:, pointIndex]

3-element Vector{Float64}:
 18.08000087738037
  4.760000109672546
 27.280001640319824

In [11]:
positions(frame)[:, pointIndex] .+= [1.0, 3.0, 1.0]

3-element view(::Chemfiles.ChemfilesArray, :, 1) with eltype Float64:
 19.08000087738037
  7.760000109672546
 28.280001640319824

In [12]:
point = positions(frame)[:, pointIndex];

In [13]:
distanceVector2 = computeDistanceVector(point, positions(frame), box);

In [14]:
distanceMatrix[pointIndex, :] = distanceVector2
distanceMatrix[:, pointIndex] = distanceVector2

512-element Vector{Float64}:
  0.0
  9.323734012019308
 15.944260634678791
 14.793398203066541
 13.395224984551355
  6.86141381803289
 12.693309954643011
 10.455247908232364
 18.502598188642576
 16.510286278632403
  9.607518721043
 14.881257478894032
 14.645831339861829
  ⋮
 15.862367411961147
 15.700268218658977
 21.41198464553139
 19.19659908280115
 16.943072847426997
 10.984066301256382
 11.132987830312294
 10.438580466801229
 17.18246264372193
  6.817903806990695
 19.098409989080544
 21.142525625335754

In [15]:
G2MatrixUpdated = buildG2Matrix(distanceMatrix, NNParms.G2Functions);

In [16]:
sum(G2MatrixUpdated .- G2Matrix)

6.734293944636168

In [17]:
sum(G2MatrixUpdated[pointIndex, :] .- G2Matrix[pointIndex, :])

3.367146972318084

In [18]:
G2MatrixOriginal = copy(G2Matrix);

In [19]:
G2Matrix == G2MatrixOriginal

true

In [20]:
updateG2Matrix!(G2Matrix, distanceVector1, distanceVector2, systemParms, NNParms.G2Functions, pointIndex);

In [21]:
G2Matrix == G2MatrixOriginal

false

In [22]:
sum(G2MatrixUpdated .- G2MatrixOriginal)

6.734293944636168

In [23]:
sum(G2MatrixUpdated .- G2Matrix)

-3.58046925441613e-15

In [26]:
@btime updateG2Matrix!($G2Matrix, $distanceVector1, $distanceVector2, $systemParms, $NNParms.G2Functions, $pointIndex);

  65.936 μs (0 allocations: 0 bytes)


In [24]:
updateG2Matrix!(G2Matrix, distanceVector2, distanceVector1, systemParms, NNParms.G2Functions, pointIndex);

In [25]:
sum(G2MatrixOriginal .- G2Matrix)

2.498001805406602e-16

In [39]:
G2MatrixOriginal[31, :] .- G2Matrix[31, :]

12-element Vector{Float64}:
 -0.03976933722389564
 -0.01915880572800721
 -0.0028843643786344852
 -0.000192889985448641
 -0.007778179461118695
 -0.0014027017915146933
 -4.5618409220127276e-5
 -1.5691500876435782e-9
 -0.033737549851149407
 -0.026389825102947517
 -0.01614665096152279
 -0.003698454612707791

In [40]:
computeG2Element(0.0, 0.003, 6.0, 1.0)

0.997004495503373

In [6]:
model

Chain(
  Dense(20 => 20, relu; bias=false),    # 400 parameters
  Dense(20 => 20, relu; bias=false),    # 400 parameters
  Dense(20 => 1; bias=false),           # 20 parameters
)                   # Total: 3 arrays, 820 parameters, 6.664 KiB.

In [65]:
inputdata = rand(Float64, (512, 14));
inputlayer = inputdata[1, :];

In [67]:
@btime atomicEnergy($inputlayer, $model)

  275.132 ns (6 allocations: 1.00 KiB)


0.296728100444809

In [92]:
@btime totalEnergyScalar(inputdata, model)

  154.099 μs (3585 allocations: 600.02 KiB)


387.64894878504174

In [87]:
atomicEnergy(inputdata[1, :], model)

0.296728100444809

In [7]:
energyGradients = computeEnergyGradients(G2Matrix, model)

3-element Vector{Matrix{Float64}}:
 [2415.920543679994 2547.4565519338016 … 2191.36532301734 1912.3621881613617; 0.0 0.0 … 0.0 0.0; … ; 3034.7183653662732 3199.945132032516 … 2752.6470638218057 2402.181921432444; 0.0 0.0 … 0.0 0.0]
 [-1627.2656754914422 -0.0 … -1116.289223062292 -0.0; 2578.3771711348572 0.0 … 1768.7429240824147 0.0; … ; 1373.4965021267865 0.0 … 942.2059140864284 0.0; 0.0 0.0 … 0.0 0.0]
 [5444.763771149569 7445.66566094606 … 9993.053626551657 0.0]

In [8]:
@btime computeEnergyGradients($G2Matrix, $model)

  6.449 ms (20057 allocations: 88.38 MiB)


3-element Vector{Matrix{Float64}}:
 [2415.920543679994 2547.4565519338016 … 2191.36532301734 1912.3621881613617; 0.0 0.0 … 0.0 0.0; … ; 3034.7183653662732 3199.945132032516 … 2752.6470638218057 2402.181921432444; 0.0 0.0 … 0.0 0.0]
 [-1627.2656754914422 -0.0 … -1116.289223062292 -0.0; 2578.3771711348572 0.0 … 1768.7429240824147 0.0; … ; 1373.4965021267865 0.0 … 942.2059140864284 0.0; 0.0 0.0 … 0.0 0.0]
 [5444.763771149569 7445.66566094606 … 9993.053626551657 0.0]

In [5]:
cutoff = 6.0 # Å
rs = 0.0 # Å 
eta = 0.1 # Å^-2
lambda = 1.0
zeta = 1.0;

In [6]:
distanceVector = distanceMatrix[:, 1];

In [7]:
function computeCosAngle(coordinates, i, j, k, distance_ij, distance_ik)::Float64
    @assert i != j && i != k && k != j 
    vector_0i = coordinates[:, i]
    vector_ij = coordinates[:, j] .- vector_0i
    vector_ik = coordinates[:, k] .- vector_0i
    cosAngle = dot(vector_ij, vector_ik) / (distance_ij * distance_ik)
    return (cosAngle)
end

computeCosAngle (generic function with 1 method)

In [8]:
@btime computeCosAngle($coordinates, 1, 461, 483, distanceVector[461], distanceVector[483])

  142.165 ns (9 allocations: 480 bytes)


0.7169325422695667

In [9]:
function G9(cosAngle, distance_ij, distance_ik, cutoff, eta, zeta, lambda=1.0, rshift=0.0)::Float64
    return (
        (1.0 + lambda * cosAngle)^zeta * 
        exp(-eta * (
                (distance_ij - rshift)^2 + 
                (distance_ik - rshift)^2)
                ) * 
        distanceCutoff(distance_ij, cutoff) * 
        distanceCutoff(distance_ik, cutoff))
end        

G9 (generic function with 3 methods)

In [10]:
function G3(cosAngle, distance_ij, distance_ik, distance_kj, cutoff, eta, zeta, lambda=1.0, rshift=0.0)::Float64
    return (
        (1.0 + lambda * cosAngle)^zeta * 
        exp(-eta * (
                (distance_ij - rshift)^2 + 
                (distance_ik - rshift)^2 +
                (distance_kj - rshift)^2)) * 
        distanceCutoff(distance_ij, cutoff) * 
        distanceCutoff(distance_ik, cutoff) *
        distanceCutoff(distance_kj, cutoff))
end        

G3 (generic function with 3 methods)

In [11]:
function G9total(i, coordinates, distanceMatrix, cutoff, eta, zeta, lambda=1.0, rshift=0.0)::Float64
    sum = 0.0
    distanceVector = distanceMatrix[:, i];
    N = length(distanceVector)
    @inbounds for k in eachindex(distanceVector)
        distance_ik = distanceVector[k]
        @inbounds for j in 1:k-1
            distance_ij = distanceVector[j]
            if 0 < distance_ij < cutoff && 0 < distance_ik < cutoff
                cosAngle = computeCosAngle(coordinates, i, j, k, distance_ij, distance_ik)
                sum += G9(cosAngle, distance_ij, distance_ik, cutoff, eta, zeta, lambda, rshift)
            end
        end
    end
    return (2.0^(1.0 - zeta) * sum)
end

G9total (generic function with 3 methods)

In [12]:
@btime G9total(5, $coordinates, $distanceMatrix, cutoff, eta, 1.0, 1.0)

  11.758 μs (333 allocations: 29.95 KiB)


0.6491223630240601

In [13]:
G9total(5, coordinates, distanceMatrix, cutoff, eta, 1.0, 1.0)

0.6491223630240601

I will implement the G3total function later (because an additional distance has to be computed)

In [15]:
@btime buildDistanceMatrix_old($frame);

  6.737 ms (2 allocations: 2.00 MiB)


In [16]:
@btime buildDistanceMatrix($frame);

  5.273 ms (3080 allocations: 12.27 MiB)


In [17]:
distanceMatrix1 = buildDistanceMatrix_old(frame)
distanceMatrix2 = buildDistanceMatrix(frame)
distanceMatrix1 == distanceMatrix2

true

In [18]:
distanceVector = distanceMatrix[:, 1];

In [19]:
@btime $distanceMatrix[:, $1];

  435.050 ns (1 allocation: 4.12 KiB)


In [31]:
coordinates = positions(frame);
r1 = coordinates[:, 1]
r2 = coordinates[:, 2]

3-element Vector{Float64}:
 15.830000638961792
 32.47000217437744
 31.760001182556152

In [35]:
distanceVector1 = distanceMatrix[:, 2];

In [37]:
distanceVector2 = computeDistanceVector(r2, coordinates, box);

In [33]:
distanceVector1 == distanceVector2

true

In [40]:
@btime updateDistance!($frame, $distanceVector, $1);

  10.216 μs (0 allocations: 0 bytes)


In [41]:
@btime computeDistanceVector($r1, $coordinates, $box);

  6.400 μs (5 allocations: 20.45 KiB)
